# Aula09.Ex01 - Exemplo - Esfera - Transformação Geométrica 3D

### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [20]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

### Inicializando janela

In [21]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(700, 700, "Esfera", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec2.
* Definindo uma variável chamada mat_transformation do tipo mat4 (matriz 4x4). Use ela como matriz de transformação, resultante de uma sequências de outras transformações (e.g. rotação + translação)
* Usamos vec2, pois nosso programa (na CPU) irá enviar apenas duas coordenadas para plotar um ponto. Podemos mandar três coordenadas (vec3) e até mesmo quatro coordenadas (vec4).
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base em uma matriz de transformação, conforme estudado na Aula05.

In [22]:
vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,1.0);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [23]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [24]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [25]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [26]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [27]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [28]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [29]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Modelando uma Esfera

In [30]:
import math


PI = 3.141592
r = 0.5 # raio
num_sectors = 30 # qtd de sectors (longitude)
num_stacks = 30 # qtd de stacks (latitude)

# grid sectos vs stacks (longitude vs latitude)
sector_step=(PI*2)/num_sectors # variar de 0 até 2π
stack_step=(PI)/num_stacks # variar de 0 até π

# Entrada: angulo de longitude, latitude, raio
# Saida: coordenadas na esfea
def F(u,v,r):
    x = r*math.sin(v)*math.cos(u)
    y = r*math.sin(v)*math.sin(u)
    z = r*math.cos(v)
    return (x,y,z)

# vamos gerar um conjunto de vertices representantes poligonos
# para a superficie da esfera.
# cada poligono eh representado por dois triangulos
vertices_list = []
for i in range(0,num_sectors): # para cada sector (longitude)
    
    for j in range(0,num_stacks): # para cada stack (latitude)
        
        
        
        u = i * sector_step # angulo setor
        v = j * stack_step # angulo stack
        
        un = 0 # angulo do proximo sector
        if i+1==num_sectors:
            un = PI*2
        else: un = (i+1)*sector_step
            
        vn = 0 # angulo do proximo stack
        if j+1==num_stacks:
            vn = PI
        else: vn = (j+1)*stack_step
        
        # verticies do poligono
        p0=F(u, v, r)
        p1=F(u, vn, r)
        p2=F(un, v, r)
        p3=F(un, vn, r)
        
        # triangulo 1 (primeira parte do poligono)
        vertices_list.append(p0)
        vertices_list.append(p2)
        vertices_list.append(p1)
        
        # triangulo 2 (segunda e ultima parte do poligono)
        vertices_list.append(p3)
        vertices_list.append(p1)
        vertices_list.append(p2)


total_vertices = len(vertices_list)
vertices = np.zeros(total_vertices, [("position", np.float32, 3)])
vertices['position'] = np.array(vertices_list)

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [31]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)


### Abaixo, nós enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [32]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [33]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.

In [34]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, nós indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [35]:
glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

### Capturando entradas do teclado
Precisamos capturar cada conjunto de teclas responsável por controlar a rotação, translação e escala em cada eixo.

In [36]:
dx = 0.0 # Rotação no eixo x
dy = 0.0 # Rotação no eixo y
dz = 0.0 # Rotação no eixo z

posx = 0.0 # Translação em x
posy = 0.0 # Translação em y
posz = 0.0 # Translação em z

sx = 1.0 # Escala em x
sy = 1.0 # Escala em y
sz = 1.0 # Escala em z


def key_event(window,key,scancode,action,mods):
    global dx, dy, dz, posx, posy, posz, sx, sy, sz
    
    # action 1 == aperta
    # action 2 == segura
    # action 0 == solta
    
    if action != 1 and action != 2:
        return;
    
    # Rotação
    if key == 65: dy += 0.05 # Segura 'A'
    elif key == 68: dy -= 0.05 # Segura 'D'
    elif key == 87: dx += 0.05 # Segura 'W'
    elif key == 83: dx -= 0.05 # Segura 'S'
    elif key == 81: dz += 0.05 # Segura 'Q'
    elif key == 69: dz -= 0.05 # Segura 'E'
    
    # Translação
    elif key == 263: posx -= 0.05 # Segura seta para esquerda
    elif key == 262: posx += 0.05 # Segura seta para direita
    elif key == 265: posy += 0.05 # Segura seta para cima
    elif key == 264: posy -= 0.05 # Segura seta para baixo
    elif key == 79: posz -= 0.05 # Segura 'O'
    elif key == 76: posz += 0.05 # Segura 'L'
        
    # Escala
    elif key == 88: sx -= 0.05 # Segura 'X'
    elif key == 67: sx += 0.05 # Segura 'C'
    elif key == 86: sy -= 0.05 # Segura 'V'
    elif key == 66: sy += 0.05 # Segura 'B'
    elif key == 78: sz -= 0.05 # Segura 'N'
    elif key == 77: sz += 0.05 # Segura 'M'
    elif key == 45: # Segura '-'
        sx -= 0.05
        sy -= 0.05
        sz -= 0.05
    elif key == 61: # Segura '+'
        sx += 0.05
        sy += 0.05
        sz += 0.05
    
        
    #print("key = %s\tscancode = %s\taction = %s" % (key, scancode, action))
    
glfw.set_key_callback(window,key_event)

###  Vamos pegar a localização da variável color (uniform) do Fragment Shader para que possamos alterá-la em nosso laço da janela!

In [37]:
loc_color = glGetUniformLocation(program, "color")

### Nesse momento, nós exibimos a janela!


In [38]:
glfw.show_window(window)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.


Usaremos o GL_TRIANGLE para modelar os triângulos (que formarão outros polígonos) da superfície da nossa esfera.

In [39]:
import math
import random as rand

glEnable(GL_DEPTH_TEST) ### importante para 3D

def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    # Agora a esfera rotaciona em cada eixo separado usando as setas do teclado.
    
    cos_dx = math.cos(dx)
    sin_dx = math.sin(dx)
    
    cos_dy = math.cos(dy)
    sin_dy = math.sin(dy)
    
    cos_dz = math.cos(dz)
    sin_dz = math.sin(dz)
    
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    mat_rotation_z = np.array([     cos_dz, -sin_dz, 0.0, 0.0, 
                                    sin_dz,  cos_dz, 0.0, 0.0, 
                                    0.0,      0.0, 1.0, 0.0, 
                                    0.0,      0.0, 0.0, 1.0], np.float32)
    
    mat_rotation_x = np.array([     1.0,   0.0,    0.0, 0.0, 
                                    0.0, cos_dx, -sin_dx, 0.0, 
                                    0.0, sin_dx,  cos_dx, 0.0, 
                                    0.0,   0.0,    0.0, 1.0], np.float32)
    
    mat_rotation_y = np.array([     cos_dy,  0.0, sin_dy, 0.0, 
                                    0.0,    1.0,   0.0, 0.0, 
                                    -sin_dy, 0.0, cos_dy, 0.0, 
                                    0.0,    0.0,   0.0, 1.0], np.float32)
    
    mat_scale = np.array([          sx,  0.0, 0.0, 0.0, 
                                    0.0, sy,  0.0, 0.0, 
                                    0.0, 0.0, sz,  0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)
    
    mat_translation  = np.array([   1.0,  0.0, 0.0, posx, 
                                    0.0,  1.0, 0.0, posy, 
                                    0.0,  0.0, 1.0, posz, 
                                    0.0,  0.0, 0.0,  1.0], np.float32)
    
    mat_transform = multiplica_matriz(mat_rotation_z,mat_rotation_x)
    mat_transform = multiplica_matriz(mat_rotation_y,mat_transform)
    mat_transform = multiplica_matriz(mat_scale,mat_transform)
    mat_transform = multiplica_matriz(mat_translation,mat_transform)

    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)

    #glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    
    for triangle in range(0,len(vertices),3):
       
        rand.seed( triangle )
        R = rand.random()
        G = rand.random()
        B = rand.random()        
        glUniform4f(loc_color, R, G, B, 1.0)
        
        glDrawArrays(GL_TRIANGLES, triangle, 3)     

    
    glfw.swap_buffers(window)

glfw.terminate()

# Exercício

Modifique esse código para aplicar outras transformações (de sua escolha) conforme eventos do teclado.